# Convert Raster Data to DGGS Zones

In [1]:
import os.path

import folium
import json
import pystac_client
import shapely
from tqdm.autonotebook import tqdm

from vgrid.conversion.raster2dggs import raster2a5, raster2rhealpix, raster2h3
from vgrid.utils.io import convert_to_output_format, download_file

/tmp/ipykernel_386257/1827741114.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# [optional] extra dependencies for nicer notebook progress bars if not available
!pip install -U tqdm ipywidgets jupyterlab_widgets

## Identify the Study Area of Interest

In [3]:
with open("../manitoba_study_area/inputs/manitoba_study_area.geojson") as f:
    data = f.read()
    poly = json.loads(data)
    geom = shapely.from_geojson(data)

map = folium.Map(location=[geom.centroid.y, geom.centroid.x], zoom_start=8)
folium.GeoJson(poly).add_to(map)
map

## Find RCM ARD Data from EODMS matching AOI

In [4]:
client = pystac_client.Client.open("https://www.eodms-sgdot.nrcan-rncan.gc.ca/stac/")
result = client.search(
    collections=["rcm-ard"],
    intersects=geom,
    datetime=["2018-01-01", "2026-01-01"],
    limit=100,
)
result.matched()

74

In [5]:
item = next(result.items())
item

<Item id=9b27638f-6956-48e1-b66e-8f82df5cbbf5>

## Convert RCM ARD Data to DGGS

#### NOTE

⚠️ Requires patch: https://github.com/opengeoshub/vgrid/pull/57


In [7]:
output_formats = ["geojson", "parquet"]
dggs_types_res = [
    ("H3", raster2h3, [7, 8]),  # H3 L8 hexagon dimension ~= ISEA7H L9
    # ("A5", raster2a5, [9]),
    # ("rHEALPix", raster2rhealpix, [8, 9]),
]

dggs_bar = tqdm(dggs_types_res, desc="Processing DGGS Types")
for dggrs, dggrs_builder, dggrs_resolutions in dggs_bar:
    dggs_bar.set_postfix(dggrs=dggrs)
    res_bar = tqdm(dggrs_resolutions, desc="Processing Resolutions", leave=False, total=len(dggrs_resolutions))
    for resolution in res_bar:
        res_bar.set_postfix(dggrs=dggrs, resolution=resolution)
        stac_items_bar = tqdm(result.items(), desc="Processing STAC Items", leave=False, total=result.matched())
        for item in stac_items_bar:
            stac_items_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=item.id)
            date = item.properties["datetime"][:10]  # YYYY-MM-DD
            for band in ["rl", "rr"]:
                src_uri = item.assets[band].href
                src_name = os.path.basename(src_uri)
                res_bar.set_postfix(dggrs=dggrs, resolution=resolution, item=src_name)

                src_dir = os.path.join("./outputs", f"manitoba_rcm_ard", "src", date, band)
                src_path = os.path.join(src_dir, src_name)
                out_dir = os.path.join("./outputs", "manitoba_rcm_ard", dggrs, f"L{resolution}", date, band)
                os.makedirs(out_dir, exist_ok=True)
                out_files = {
                    fmt: os.path.join(out_dir, f"{os.path.splitext(src_name)[0]}.{fmt}")
                    for fmt in output_formats
                }
                if all(os.path.isfile(p) for p in out_files.values()):
                    print(f"  Skipping... All output files already exist: {list(out_files.values())}")
                    continue

                print(f"Converting to {dggrs}-L{resolution} [band={band}]. Saving to [{out_dir}]...")
                if not os.path.isfile(src_path):
                    os.makedirs(src_dir, exist_ok=True)
                    download_file(src_uri, src_path)
                out_df = dggrs_builder(
                    src_path,
                    resolution=resolution,
                    output_format="gpd"  # GeoPandas Dataframe
                )
                # save results
                for fmt, out_path in out_files.items():
                    convert_to_output_format(out_df, output_format=fmt, output_name=out_path)

Processing DGGS Types:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Resolutions:   0%|          | 0/2 [00:00<?, ?it/s]

Processing STAC Items:   0%|          | 0/74 [00:00<?, ?it/s]

Converting to H3-L7 [band=rl]. Saving to [./outputs/manitoba_rcm_ard/H3/L7/2025-08-20/rl]...





Converting raster to H3:   0%|          | 0/4327 [00:00<?, ? cells/s]


Converting raster to H3:   8%|▊         | 347/4327 [00:00<00:01, 3429.58 cells/s]


Converting raster to H3:  16%|█▌        | 690/4327 [00:00<00:01, 3417.01 cells/s]


Converting raster to H3:  24%|██▍       | 1032/4327 [00:00<00:01, 3172.36 cells/s]


Converting raster to H3:  31%|███▏      | 1358/4327 [00:00<00:00, 3204.19 cells/s]


Converting raster to H3:  41%|████      | 1769/4327 [00:00<00:00, 3519.31 cells/s]


Converting raster to H3:  49%|████▉     | 2128/4327 [00:00<00:00, 3542.01 cells/s]


Converting raster to H3:  59%|█████▉    | 2550/4327 [00:00<00:00, 3759.36 cells/s]


Converting raster to H3:  68%|██████▊   | 2947/4327 [00:00<00:00, 3823.83 cells/s]


Converting raster to H3:  77%|███████▋  | 3331/4327 [00:00<00:00, 3650.65 cells/s]


Converting raster to H3:  86%|████████▌ | 3726/4327 [00:01<00:00, 3738.03 cells/s]


Converting raster to H3: 100%|██████████| 4327/4327 [00:01<00:00, 3621.76 cel

...

Downloading...
From: https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2025/07/29/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_CH_CV_MLC/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RL.tif
To: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/outputs/manitoba_rcm_ard/src/2025-07-29/rl/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RL.tif



  0%|          | 0.00/121M [00:00<?, ?B/s]


  0%|          | 524k/121M [00:00<00:58, 2.06MB/s]


  1%|          | 1.05M/121M [00:00<00:44, 2.68MB/s]


  2%|▏         | 2.10M/121M [00:00<00:32, 3.63MB/s]


  2%|▏         | 2.62M/121M [00:00<00:29, 3.95MB/s]


  3%|▎         | 3.67M/121M [00:00<00:23, 5.05MB/s]


  4%|▍         | 4.72M/121M [00:01<00:21, 5.40MB/s]


  5%|▍         | 5.77M/121M [00:01<00:21, 5.44MB/s]


  6%|▌         | 6.82M/121M [00:01<00:19, 5.80MB/s]


  6%|▋         | 7.86M/121M [00:01<00:19, 5.70MB/s]


  7%|▋         | 8.91M/121M [00:01<00:17, 6.37MB/s]


  8%|▊         | 9.96M/121M [00:01<00:16

Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L7/2025-07-29/rl/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RL.geojson
Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L7/2025-07-29/rl/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RL.parquet
Converting to H3-L7 [band=rr]. Saving to [./outputs/manitoba_rcm_ard/H3/L7/2025-07-29/rr]...


Downloading...
From: https://rcm-ceos-ard.s3.ca-central-1.amazonaws.com/MLC/2025/07/29/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_CH_CV_MLC/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RR.tif
To: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/outputs/manitoba_rcm_ard/src/2025-07-29/rr/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RR.tif



  0%|          | 0.00/121M [00:00<?, ?B/s]


  1%|          | 1.05M/121M [00:00<00:12, 9.56MB/s]


  2%|▏         | 2.62M/121M [00:00<00:10, 11.6MB/s]


  3%|▎         | 4.19M/121M [00:00<00:09, 12.7MB/s]


  5%|▍         | 5.77M/121M [00:00<00:08, 13.3MB/s]


  6%|▌         | 7.34M/121M [00:00<00:08, 13.6MB/s]


  7%|▋         | 8.91M/121M [00:00<00:08, 13.3MB/s]


  9%|▊         | 10.5M/121M [00:00<00:08, 12.5MB/s]


 10%|█         | 12.6M/121M [00:00<00:07, 14.0MB/s]


 12%|█▏        | 14.2M/121M [00:01<00:08, 12.4MB/s]


 13%|█▎        | 16.3M/121M [00:01<00:07, 13.9MB/s]


 15%|█▍        | 17.8M/121M [00:01<00:0

Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L7/2025-07-29/rr/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RR.geojson
Output file saved as: /home/francis/dev/ogc/ogc-dggs/ogc11-dggs/manitoba-rcm-ard/./outputs/manitoba_rcm_ard/H3/L7/2025-07-29/rr/RCM1_OK3556292_PK3720548_1_SC30MCPD_20250729_002140_RR.parquet


APIError: {"detail":"Method Not Allowed"}